# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,18.86,74,100,9.14,PN,1694911475
1,1,tralee,52.2704,-9.7026,12.04,91,100,2.21,IE,1694911476
2,2,port-aux-francais,-49.3500,70.2167,5.70,96,100,15.29,TF,1694911476
3,3,bathsheba,13.2167,-59.5167,28.34,78,20,1.03,BB,1694911477
4,4,quesnel,52.9999,-122.5029,18.28,63,0,0.00,CA,1694911209


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    &(city_data_df["Wind Speed"] < 4.5) \
    &(city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
67,67,tinghir,31.5157,-5.5343,22.78,30,0,2.81,MA,1694911505
118,118,tabas,33.5959,56.9244,24.21,22,0,3.41,IR,1694911528
127,127,rawlins,41.7911,-107.2387,22.94,16,0,3.60,US,1694911383
137,137,sao vicente,-23.9631,-46.3919,26.26,26,0,1.47,BR,1694911535
143,143,andalgala,-27.5819,-66.3165,24.39,15,0,1.17,AR,1694911538
148,148,westport,41.1415,-73.3579,22.08,72,0,2.57,US,1694910962
154,154,saint-paul,-21.0096,55.2707,21.11,88,0,0.45,RE,1694911534
248,248,fort mcmurray,56.7268,-111.3810,21.77,40,0,3.09,CA,1694911330
266,266,hwange,-18.3693,26.5019,21.85,33,0,0.80,ZW,1694911595
313,313,ballina,-28.8667,153.5667,23.20,53,0,4.42,AU,1694911486


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
67,tinghir,MA,31.5157,-5.5343,30,
118,tabas,IR,33.5959,56.9244,22,
127,rawlins,US,41.7911,-107.2387,16,
137,sao vicente,BR,-23.9631,-46.3919,26,
143,andalgala,AR,-27.5819,-66.3165,15,
148,westport,US,41.1415,-73.3579,72,
154,saint-paul,RE,-21.0096,55.2707,88,
248,fort mcmurray,CA,56.7268,-111.3810,40,
266,hwange,ZW,-18.3693,26.5019,33,
313,ballina,AU,-28.8667,153.5667,53,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"Proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tinghir - nearest hotel: No hotel found
tabas - nearest hotel: No hotel found
rawlins - nearest hotel: No hotel found
sao vicente - nearest hotel: No hotel found
andalgala - nearest hotel: No hotel found
westport - nearest hotel: No hotel found
saint-paul - nearest hotel: No hotel found
fort mcmurray - nearest hotel: No hotel found
hwange - nearest hotel: No hotel found


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    hover_cols=["Hotel", "Country"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)